In [ ]:
# !pip install optuna
# !pip install -U xgboost

In [ ]:
# =====================================================
# XGBoost Model with Optuna Hyperparameter Optimization
# =====================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from xgboost.callback import EarlyStopping
from xgboost import XGBClassifier
from scipy.stats import mode
import optuna

# ======================
# 1. Load Data
# ======================
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

# Drop id if exists
if 'id' in train.columns:
    train.drop('id', axis=1, inplace=True)
if 'id' in test.columns:
    test_id = test['id']
    test.drop('id', axis=1, inplace=True)
else:
    test_id = pd.Series(range(len(test)), name='id')

# Split target and features
X_train = train.drop('WeightCategory', axis=1)
y_train = train['WeightCategory']
X_test = test

# ======================
# 2. One-Hot Encode categorical columns
# ======================
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()
numeric_cols = X_train.select_dtypes(exclude='object').columns.tolist()

ohe = OneHotEncoder(drop=None, sparse_output=False, handle_unknown='ignore')

X_train_cat_encoded = ohe.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = ohe.transform(X_test[categorical_cols])

X_train_cat_encoded = pd.DataFrame(
    X_train_cat_encoded,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat_encoded = pd.DataFrame(
    X_test_cat_encoded,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

X_train_encoded = pd.concat([X_train[numeric_cols], X_train_cat_encoded], axis=1)
X_test_encoded = pd.concat([X_test[numeric_cols], X_test_cat_encoded], axis=1)

# ======================
# 3. Encode target labels
# ======================
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

print(f"\nEncoded Train Shape: {X_train_encoded.shape}")
print(f"Encoded Test Shape: {X_test_encoded.shape}")

# ======================
# 4. Define Optuna Objective
# ======================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.5, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "subsample": trial.suggest_float("subsample", 0.9, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.9, 1.0),
        "random_state": 42,
        "use_label_encoder": False,
        "eval_metric": "mlogloss",
        "n_jobs": -1,
    }

    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    model = XGBClassifier(**params)

    scores = cross_val_score(model, X_train_encoded, y_train_encoded,
                             cv=kf, scoring=make_scorer(accuracy_score))
    return scores.mean()

# ======================
# 5. Run Optuna Study
# ======================
print("\n Running Optuna optimization (this may take a few minutes)...")
opt = optuna.create_study(direction="maximize")
opt.optimize(objective, n_trials=130, show_progress_bar=True)

print("\n===== Optuna Optimization Results =====")
print(f"Best CV Accuracy: {opt.best_value:.4f}")
print("Best Parameters Found:")
for k, v in opt.best_params.items():
    print(f"  {k}: {v}")

# Save all trial results
opt.trials_dataframe().to_csv("optuna_xgb_results.csv", index=False)
print("\n Saved all Optuna trials to 'optuna_xgb_results.csv'")

# ======================
# 6. Use Best Parameters
# ======================
best_params = opt.best_params
best_params.update({
    "random_state": 42,
    "use_label_encoder": False,
    "eval_metric": "mlogloss",
    "n_jobs": -1
})

print("\n===== Using Best Parameters for Final Training =====")
print(best_params)

# ======================
# 7. 5-Fold Cross Validation Training
# ======================
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
test_preds = np.zeros((X_test_encoded.shape[0], kf.n_splits))
val_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_encoded, y_train_encoded)):
    print(f"\n===== Fold {fold + 1} =====")
    X_tr, X_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_tr, y_val = y_train_encoded[train_idx], y_train_encoded[val_idx]

    model = XGBClassifier(**best_params)
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

    val_pred = model.predict(X_val)
    acc = accuracy_score(y_val, val_pred)
    val_scores.append(acc)
    print(f"Validation Accuracy: {acc:.4f}")

    test_preds[:, fold] = model.predict(X_test_encoded)

# ======================
# 8. Ensemble Test Predictions
# ======================
if len(np.unique(y_train_encoded)) > 2:
    final_preds = mode(test_preds, axis=1)[0].astype(int).ravel()
else:
    final_preds = np.round(test_preds.mean(axis=1)).astype(int)

print("\n===== Final Results =====")
print(f"Mean CV Accuracy: {np.mean(val_scores):.4f}")
print(f"Std Dev: {np.std(val_scores):.4f}")

# Decode predictions
final_labels = label_encoder.inverse_transform(final_preds)

# ======================
# 9. Create Submission File
# ======================
submission = pd.DataFrame({
    'id': test_id,
    'WeightCategory': final_labels
})

print("\n Prediction Preview:")
print(submission.head())
print(f"\nTotal rows: {len(submission)}")

submission.to_csv("predictions.csv", index=False)
print("\n Saved 'predictions.csv' — ready for Kaggle upload!")


[I 2025-10-26 12:49:28,614] A new study created in memory with name: no-name-3b302836-b38d-40b7-86e6-160c86efab7d


Train Shape: (15533, 18)
Test Shape: (5225, 17)

Encoded Train Shape: (15533, 30)
Encoded Test Shape: (5225, 30)

🔍 Running Optuna optimization (this may take a few minutes)...


  0%|          | 0/130 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:49:28] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:50:04] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:50:43] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-26 12:51:21,143] Trial 0 finished with value: 0.8963494533752366 and parameters: {'n_estimators': 2174, 'learning_rate': 0.07827931407302348, 'max_depth': 5, 'subsample': 0.9124199470938431, 'colsample_bytree': 0.9410604548429887}. Best is trial 0 with value: 0.8963494533752366.


...

Validation Accuracy: 0.9079

===== Fold 4 =====


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:33:58] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Validation Accuracy: 0.9089

===== Fold 5 =====


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:34:25] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Validation Accuracy: 0.9047

===== Final Results =====
Mean CV Accuracy: 0.9063
Std Dev: 0.0017

✅ Prediction Preview:
      id       WeightCategory
0  15533     Obesity_Type_III
1  15534   Overweight_Level_I
2  15535  Overweight_Level_II
3  15536      Obesity_Type_II
4  15537        Normal_Weight

Total rows: 5225

✅ Saved 'predictions.csv' — ready for Kaggle upload!


In [ ]:
# [I 2025-10-26 12:49:28,614] A new study created in memory with name: no-name-3b302836-b38d-40b7-86e6-160c86efab7d
# Train Shape: (15533, 18)
# Test Shape: (5225, 17)

# Encoded Train Shape: (15533, 30)
# Encoded Test Shape: (5225, 30)

# 🔍 Running Optuna optimization (this may take a few minutes)...
# Best trial: 125. Best value: 0.906071: 100%
#  130/130 [2:43:09<00:00, 56.87s/it]
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:49:28] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:50:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:50:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 12:51:21,143] Trial 0 finished with value: 0.8963494533752366 and parameters: {'n_estimators': 2174, 'learning_rate': 0.07827931407302348, 'max_depth': 5, 'subsample': 0.9124199470938431, 'colsample_bytree': 0.9410604548429887}. Best is trial 0 with value: 0.8963494533752366.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:51:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:51:55] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:52:29] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 12:53:05,383] Trial 1 finished with value: 0.8924867965510561 and parameters: {'n_estimators': 2868, 'learning_rate': 0.15432676811194615, 'max_depth': 3, 'subsample': 0.9764709476150247, 'colsample_bytree': 0.9311269948467077}. Best is trial 0 with value: 0.8963494533752366.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:53:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:53:44] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:54:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 12:55:13,390] Trial 2 finished with value: 0.8953837798430974 and parameters: {'n_estimators': 1990, 'learning_rate': 0.075996290331253, 'max_depth': 8, 'subsample': 0.9615988428592805, 'colsample_bytree': 0.9667703109635043}. Best is trial 0 with value: 0.8963494533752366.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:55:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:55:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:56:07] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 12:56:32,567] Trial 3 finished with value: 0.8989890115979059 and parameters: {'n_estimators': 1115, 'learning_rate': 0.04976988451522882, 'max_depth': 8, 'subsample': 0.9439112308052658, 'colsample_bytree': 0.9384592284884457}. Best is trial 3 with value: 0.8989890115979059.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:56:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:57:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:57:51] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 12:58:29,878] Trial 4 finished with value: 0.9033669164741823 and parameters: {'n_estimators': 1721, 'learning_rate': 0.00583960001143491, 'max_depth': 6, 'subsample': 0.9326221157121933, 'colsample_bytree': 0.9016313369037028}. Best is trial 4 with value: 0.9033669164741823.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:58:29] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:59:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:00:19] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:01:15,872] Trial 5 finished with value: 0.89725072711826 and parameters: {'n_estimators': 2827, 'learning_rate': 0.051277917679284454, 'max_depth': 7, 'subsample': 0.9217678198775279, 'colsample_bytree': 0.9628929616769976}. Best is trial 4 with value: 0.9033669164741823.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:01:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:01:28] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:01:41] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:01:54,288] Trial 6 finished with value: 0.8913924726581922 and parameters: {'n_estimators': 690, 'learning_rate': 0.005370840332473933, 'max_depth': 5, 'subsample': 0.960634114376427, 'colsample_bytree': 0.9274618597740849}. Best is trial 4 with value: 0.9033669164741823.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:01:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:02:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:03:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:04:09,009] Trial 7 finished with value: 0.8973795640008686 and parameters: {'n_estimators': 2609, 'learning_rate': 0.050604173636976287, 'max_depth': 5, 'subsample': 0.9221281486429068, 'colsample_bytree': 0.9057806819717397}. Best is trial 4 with value: 0.9033669164741823.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:04:09] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:04:44] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:05:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:05:48,690] Trial 8 finished with value: 0.904203889906428 and parameters: {'n_estimators': 1850, 'learning_rate': 0.005049029878415169, 'max_depth': 5, 'subsample': 0.9212812875390192, 'colsample_bytree': 0.9242734939821045}. Best is trial 8 with value: 0.904203889906428.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:05:48] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:06:22] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:06:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:07:24,838] Trial 9 finished with value: 0.9051051263450747 and parameters: {'n_estimators': 2254, 'learning_rate': 0.011187083208623367, 'max_depth': 4, 'subsample': 0.9208194631517832, 'colsample_bytree': 0.943926930411497}. Best is trial 9 with value: 0.9051051263450747.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:07:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:07:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:07:55] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:08:10,076] Trial 10 finished with value: 0.9027231548440766 and parameters: {'n_estimators': 1311, 'learning_rate': 0.017445802040397633, 'max_depth': 3, 'subsample': 0.997676158310647, 'colsample_bytree': 0.9793350537864343}. Best is trial 9 with value: 0.9051051263450747.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:08:10] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:08:46] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:09:17] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:09:49,644] Trial 11 finished with value: 0.9041394403781431 and parameters: {'n_estimators': 2310, 'learning_rate': 0.012841123337865301, 'max_depth': 4, 'subsample': 0.900396652354969, 'colsample_bytree': 0.995670467165784}. Best is trial 9 with value: 0.9051051263450747.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:09:49] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:10:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:10:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:10:56,913] Trial 12 finished with value: 0.905105238258205 and parameters: {'n_estimators': 1587, 'learning_rate': 0.014547683419626131, 'max_depth': 4, 'subsample': 0.9389452605585202, 'colsample_bytree': 0.9178732810916991}. Best is trial 12 with value: 0.905105238258205.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:10:56] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:11:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:11:36] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:11:54,908] Trial 13 finished with value: 0.9054271004210693 and parameters: {'n_estimators': 1418, 'learning_rate': 0.017724623708040805, 'max_depth': 4, 'subsample': 0.9426419688984071, 'colsample_bytree': 0.9168276284734445}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:11:55] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:12:17] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:12:36] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:12:58,265] Trial 14 finished with value: 0.9043969897954374 and parameters: {'n_estimators': 1450, 'learning_rate': 0.025144523935613033, 'max_depth': 4, 'subsample': 0.9444277975426222, 'colsample_bytree': 0.9158765191535705}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:12:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:13:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:13:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:13:32,987] Trial 15 finished with value: 0.8944825434044507 and parameters: {'n_estimators': 886, 'learning_rate': 0.3076925911957726, 'max_depth': 3, 'subsample': 0.9580682617991528, 'colsample_bytree': 0.9122996065098198}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:13:33] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:14:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:14:37] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:15:09,622] Trial 16 finished with value: 0.9007917083014599 and parameters: {'n_estimators': 1551, 'learning_rate': 0.029316807670455376, 'max_depth': 6, 'subsample': 0.9360801898752057, 'colsample_bytree': 0.9523996542927253}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:15:09] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:15:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:15:41] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:15:56,738] Trial 17 finished with value: 0.9017574191379759 and parameters: {'n_estimators': 1116, 'learning_rate': 0.008879106699762707, 'max_depth': 4, 'subsample': 0.9758252113246904, 'colsample_bytree': 0.9188346027495763}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:15:56] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:16:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:16:11] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:16:20,085] Trial 18 finished with value: 0.900791720736252 and parameters: {'n_estimators': 500, 'learning_rate': 0.022717244179504287, 'max_depth': 4, 'subsample': 0.9730261944132125, 'colsample_bytree': 0.9543684992887097}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:16:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:16:38] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:16:56] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:17:16,686] Trial 19 finished with value: 0.8892676899655131 and parameters: {'n_estimators': 1608, 'learning_rate': 0.4720526269088302, 'max_depth': 3, 'subsample': 0.9346498572897378, 'colsample_bytree': 0.909581730391176}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:17:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:17:42] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:18:07] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:18:35,146] Trial 20 finished with value: 0.9027875173288157 and parameters: {'n_estimators': 1289, 'learning_rate': 0.00900945403697159, 'max_depth': 6, 'subsample': 0.9492739357461328, 'colsample_bytree': 0.9327758995495649}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:18:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:19:10] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:19:42] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:20:14,971] Trial 21 finished with value: 0.9042038028628822 and parameters: {'n_estimators': 2413, 'learning_rate': 0.013034659032313486, 'max_depth': 4, 'subsample': 0.9104184241997478, 'colsample_bytree': 0.9446213817129192}. Best is trial 13 with value: 0.9054271004210693.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:20:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:20:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:21:11] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:21:40,035] Trial 22 finished with value: 0.905491487775393 and parameters: {'n_estimators': 2026, 'learning_rate': 0.011102067254161102, 'max_depth': 4, 'subsample': 0.9287959013220731, 'colsample_bytree': 0.923563122817648}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:21:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:22:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:22:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:23:16,105] Trial 23 finished with value: 0.9022724060594345 and parameters: {'n_estimators': 1885, 'learning_rate': 0.018153563141638174, 'max_depth': 5, 'subsample': 0.9294212421185631, 'colsample_bytree': 0.921419402028246}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:23:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:23:41] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:24:02] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:24:27,201] Trial 24 finished with value: 0.9037531411217858 and parameters: {'n_estimators': 2060, 'learning_rate': 0.032321786940250355, 'max_depth': 3, 'subsample': 0.9400598675705412, 'colsample_bytree': 0.9011565088953326}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:24:27] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:24:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:25:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:25:49,474] Trial 25 finished with value: 0.9034312292197524 and parameters: {'n_estimators': 1710, 'learning_rate': 0.007690255201445827, 'max_depth': 4, 'subsample': 0.9521008350444643, 'colsample_bytree': 0.9165499049329117}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:25:49] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:26:03] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:26:17] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:26:31,599] Trial 26 finished with value: 0.905169501264606 and parameters: {'n_estimators': 993, 'learning_rate': 0.01681067017635083, 'max_depth': 4, 'subsample': 0.9300095903028371, 'colsample_bytree': 0.9325760532390768}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:26:31] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:26:48] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:27:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:27:21,876] Trial 27 finished with value: 0.901499894590266 and parameters: {'n_estimators': 1018, 'learning_rate': 0.034446160515297204, 'max_depth': 5, 'subsample': 0.9293341194568587, 'colsample_bytree': 0.9345237839265084}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:27:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:27:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:27:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:27:50,918] Trial 28 finished with value: 0.9028520290310619 and parameters: {'n_estimators': 834, 'learning_rate': 0.02026653724550112, 'max_depth': 3, 'subsample': 0.908349421738686, 'colsample_bytree': 0.9252879535401668}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:27:51] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:28:26] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:28:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:29:32,254] Trial 29 finished with value: 0.9028518798135546 and parameters: {'n_estimators': 1351, 'learning_rate': 0.007683467654537647, 'max_depth': 7, 'subsample': 0.9134195996823644, 'colsample_bytree': 0.9352932561516124}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:29:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:29:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:30:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:30:35,243] Trial 30 finished with value: 0.8996972227562966 and parameters: {'n_estimators': 1191, 'learning_rate': 0.04093274558082766, 'max_depth': 5, 'subsample': 0.9526738389472779, 'colsample_bytree': 0.9470792705548823}. Best is trial 22 with value: 0.905491487775393.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:30:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:30:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:31:17] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:31:37,125] Trial 31 finished with value: 0.9057489874535184 and parameters: {'n_estimators': 1503, 'learning_rate': 0.015259147924122536, 'max_depth': 4, 'subsample': 0.9403091445257753, 'colsample_bytree': 0.9096651048015325}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:31:37] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:31:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:32:18] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:32:40,532] Trial 32 finished with value: 0.9042038028628822 and parameters: {'n_estimators': 1445, 'learning_rate': 0.010660667870025311, 'max_depth': 4, 'subsample': 0.9451321726110644, 'colsample_bytree': 0.9108029887945537}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:32:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:33:03] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:33:27] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:33:53,364] Trial 33 finished with value: 0.8981521749483751 and parameters: {'n_estimators': 2059, 'learning_rate': 0.0851444299716453, 'max_depth': 3, 'subsample': 0.9264106470395266, 'colsample_bytree': 0.9281942198806891}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:33:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:34:06] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:34:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:34:33,632] Trial 34 finished with value: 0.9040750530238194 and parameters: {'n_estimators': 938, 'learning_rate': 0.01647204461014935, 'max_depth': 4, 'subsample': 0.9680224140527675, 'colsample_bytree': 0.9090960046622484}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:34:33] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:34:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:35:17] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:35:38,131] Trial 35 finished with value: 0.8969289022597723 and parameters: {'n_estimators': 1839, 'learning_rate': 0.11390673074566585, 'max_depth': 3, 'subsample': 0.9146592906318665, 'colsample_bytree': 0.9410204956533377}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:35:38] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:35:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:36:06] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:36:20,464] Trial 36 finished with value: 0.9011135953339089 and parameters: {'n_estimators': 684, 'learning_rate': 0.024051897511477887, 'max_depth': 6, 'subsample': 0.9926109895381952, 'colsample_bytree': 0.9058711878739455}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:36:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:36:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:36:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:37:15,262] Trial 37 finished with value: 0.9033668916045978 and parameters: {'n_estimators': 1240, 'learning_rate': 0.01077038610792921, 'max_depth': 4, 'subsample': 0.9405740864700034, 'colsample_bytree': 0.9223653636673489}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:37:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:38:09] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:38:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:39:53,938] Trial 38 finished with value: 0.9033669289089746 and parameters: {'n_estimators': 2985, 'learning_rate': 0.006770389174583654, 'max_depth': 5, 'subsample': 0.9266219903081747, 'colsample_bytree': 0.9300692330335741}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:39:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:40:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:40:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:41:27,723] Trial 39 finished with value: 0.8978946006614962 and parameters: {'n_estimators': 1470, 'learning_rate': 0.06428871275235713, 'max_depth': 7, 'subsample': 0.9558617640310506, 'colsample_bytree': 0.9378537033370865}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:41:27] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:42:10] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:42:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:43:33,452] Trial 40 finished with value: 0.9014998697206815 and parameters: {'n_estimators': 2472, 'learning_rate': 0.014565326379641462, 'max_depth': 5, 'subsample': 0.9665050312096436, 'colsample_bytree': 0.9140801939132398}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:43:33] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:43:55] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:44:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:44:44,230] Trial 41 finished with value: 0.9053626881971613 and parameters: {'n_estimators': 1680, 'learning_rate': 0.015353983256144689, 'max_depth': 4, 'subsample': 0.9373396936887575, 'colsample_bytree': 0.9185113507952585}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:44:44] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:45:06] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:45:31] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:45:57,118] Trial 42 finished with value: 0.9040106034955345 and parameters: {'n_estimators': 1737, 'learning_rate': 0.02002684110640493, 'max_depth': 4, 'subsample': 0.9340155246139404, 'colsample_bytree': 0.9048328104929}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:45:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:46:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:46:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:47:21,143] Trial 43 finished with value: 0.904976289462466 and parameters: {'n_estimators': 1950, 'learning_rate': 0.009457357833651687, 'max_depth': 4, 'subsample': 0.9461404976363552, 'colsample_bytree': 0.9248191388282962}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:47:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:47:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:48:36] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:49:14,636] Trial 44 finished with value: 0.9025299679115212 and parameters: {'n_estimators': 2199, 'learning_rate': 0.012582555945053982, 'max_depth': 5, 'subsample': 0.9192430291924824, 'colsample_bytree': 0.9201374149149409}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:49:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:49:33] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:49:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:50:14,475] Trial 45 finished with value: 0.8989893100329202 and parameters: {'n_estimators': 1688, 'learning_rate': 0.006570184047301239, 'max_depth': 3, 'subsample': 0.9385249195005482, 'colsample_bytree': 0.9288516190478014}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:50:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:50:29] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:50:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:51:00,020] Trial 46 finished with value: 0.9040750530238194 and parameters: {'n_estimators': 1101, 'learning_rate': 0.02792185597224788, 'max_depth': 4, 'subsample': 0.930979406336232, 'colsample_bytree': 0.9002246464026414}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:51:00] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:51:29] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:52:10] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:52:39,985] Trial 47 finished with value: 0.9007273333819285 and parameters: {'n_estimators': 2059, 'learning_rate': 0.03929832295041751, 'max_depth': 4, 'subsample': 0.9253490505946886, 'colsample_bytree': 0.9590211518629207}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:52:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:53:19] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:53:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:54:35,789] Trial 48 finished with value: 0.9000190973539532 and parameters: {'n_estimators': 1388, 'learning_rate': 0.015412150643544165, 'max_depth': 8, 'subsample': 0.9482298444554115, 'colsample_bytree': 0.9701569651059401}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:54:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:55:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:55:33] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:56:02,388] Trial 49 finished with value: 0.8935166957852196 and parameters: {'n_estimators': 1826, 'learning_rate': 0.19467229152712767, 'max_depth': 5, 'subsample': 0.9421852997122071, 'colsample_bytree': 0.9146746667525241}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:56:02] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:56:22] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:56:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:56:57,244] Trial 50 finished with value: 0.9048476142321569 and parameters: {'n_estimators': 1544, 'learning_rate': 0.020249237963548564, 'max_depth': 3, 'subsample': 0.9178031825394624, 'colsample_bytree': 0.996938207538102}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:56:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:57:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:57:49] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:58:10,465] Trial 51 finished with value: 0.9056846249687793 and parameters: {'n_estimators': 1614, 'learning_rate': 0.014161082024644197, 'max_depth': 4, 'subsample': 0.9359103738278434, 'colsample_bytree': 0.9170980562467598}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:58:10] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:58:34] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:58:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 13:59:19,194] Trial 52 finished with value: 0.9052338761841373 and parameters: {'n_estimators': 1632, 'learning_rate': 0.011423984267401486, 'max_depth': 4, 'subsample': 0.9356711197734262, 'colsample_bytree': 0.907097298818666}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:59:19] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:59:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:00:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:00:28,055] Trial 53 finished with value: 0.9051051760842436 and parameters: {'n_estimators': 1600, 'learning_rate': 0.012434504585464608, 'max_depth': 4, 'subsample': 0.9366288106364797, 'colsample_bytree': 0.9053042286239154}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:00:28] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:00:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:01:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:01:40,216] Trial 54 finished with value: 0.9014355072359423 and parameters: {'n_estimators': 1669, 'learning_rate': 0.005745914507115468, 'max_depth': 4, 'subsample': 0.9437803662994708, 'colsample_bytree': 0.908669291478918}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:01:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:02:02] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:02:23] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:02:46,046] Trial 55 finished with value: 0.9049764013755963 and parameters: {'n_estimators': 1508, 'learning_rate': 0.01034172745443728, 'max_depth': 4, 'subsample': 0.9243573662614997, 'colsample_bytree': 0.9134395416255406}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:02:46] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:03:17] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:03:47] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:04:30,151] Trial 56 finished with value: 0.904461327410592 and parameters: {'n_estimators': 1775, 'learning_rate': 0.007864418692041434, 'max_depth': 5, 'subsample': 0.9346483482688986, 'colsample_bytree': 0.9184122072910654}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:04:30] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:04:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:05:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:05:37,774] Trial 57 finished with value: 0.9045901145540315 and parameters: {'n_estimators': 1938, 'learning_rate': 0.01292434286739034, 'max_depth': 3, 'subsample': 0.9549416741657659, 'colsample_bytree': 0.9039520437420566}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:05:37] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:05:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:06:23] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:06:48,616] Trial 58 finished with value: 0.903173679802458 and parameters: {'n_estimators': 1621, 'learning_rate': 0.025808876157497062, 'max_depth': 4, 'subsample': 0.9380149282318831, 'colsample_bytree': 0.9856815107080364}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:06:48] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:07:19] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:07:49] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:08:19,265] Trial 59 finished with value: 0.9053626384579924 and parameters: {'n_estimators': 2149, 'learning_rate': 0.008807781695843698, 'max_depth': 4, 'subsample': 0.9503940089233872, 'colsample_bytree': 0.9222337743387096}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:08:19] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:08:47] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:09:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:09:44,911] Trial 60 finished with value: 0.9000836960997453 and parameters: {'n_estimators': 2344, 'learning_rate': 0.005104936697576181, 'max_depth': 3, 'subsample': 0.9585017947750455, 'colsample_bytree': 0.9250936153633849}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:09:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:10:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:10:44] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:11:15,037] Trial 61 finished with value: 0.9043969524910608 and parameters: {'n_estimators': 2161, 'learning_rate': 0.014601295088064046, 'max_depth': 4, 'subsample': 0.9479066838106456, 'colsample_bytree': 0.9214521620756877}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:11:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:11:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:12:27] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:13:04,014] Trial 62 finished with value: 0.9047832393126255 and parameters: {'n_estimators': 2601, 'learning_rate': 0.008800484661321906, 'max_depth': 4, 'subsample': 0.9421541790587783, 'colsample_bytree': 0.9109322094822639}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:13:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:13:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:14:00] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:14:29,039] Trial 63 finished with value: 0.9051696131777364 and parameters: {'n_estimators': 2016, 'learning_rate': 0.011222267507290247, 'max_depth': 4, 'subsample': 0.9325806895833728, 'colsample_bytree': 0.9173152637015048}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:14:29] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:14:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:15:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:15:41,368] Trial 64 finished with value: 0.902272430929019 and parameters: {'n_estimators': 1415, 'learning_rate': 0.019516031451909444, 'max_depth': 5, 'subsample': 0.9531174591737208, 'colsample_bytree': 0.9074226849879532}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:15:41] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:16:11] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:16:41] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:17:15,349] Trial 65 finished with value: 0.9037531659913703 and parameters: {'n_estimators': 2138, 'learning_rate': 0.006951816782264933, 'max_depth': 4, 'subsample': 0.9617722199711125, 'colsample_bytree': 0.9122528033837083}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:17:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:17:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:18:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:18:30,980] Trial 66 finished with value: 0.9047188270887174 and parameters: {'n_estimators': 1802, 'learning_rate': 0.008933471595873302, 'max_depth': 4, 'subsample': 0.9506978150790735, 'colsample_bytree': 0.915608154604146}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:18:31] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:18:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:19:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:19:35,664] Trial 67 finished with value: 0.9044613522801765 and parameters: {'n_estimators': 1892, 'learning_rate': 0.01743514200908445, 'max_depth': 3, 'subsample': 0.9367274132095459, 'colsample_bytree': 0.9025930624818589}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:19:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:19:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:20:22] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:20:43,761] Trial 68 finished with value: 0.9019505314617776 and parameters: {'n_estimators': 1339, 'learning_rate': 0.02319559450085762, 'max_depth': 5, 'subsample': 0.9296635230896616, 'colsample_bytree': 0.9212658631686639}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:20:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:21:06] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:21:26] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:21:49,180] Trial 69 finished with value: 0.9049764262451809 and parameters: {'n_estimators': 1515, 'learning_rate': 0.013719968973624971, 'max_depth': 4, 'subsample': 0.946586255044563, 'colsample_bytree': 0.9264439003387568}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:21:49] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:22:06] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:22:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:22:43,255] Trial 70 finished with value: 0.9033669289089746 and parameters: {'n_estimators': 1209, 'learning_rate': 0.011318262049255094, 'max_depth': 4, 'subsample': 0.9408633684268678, 'colsample_bytree': 0.9177282844802733}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:22:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:23:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:23:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:24:17,870] Trial 71 finished with value: 0.904654539212732 and parameters: {'n_estimators': 2255, 'learning_rate': 0.011380540238081533, 'max_depth': 4, 'subsample': 0.932914347637374, 'colsample_bytree': 0.9168788673666962}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:24:18] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:24:47] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:25:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:25:42,718] Trial 72 finished with value: 0.9051051885190359 and parameters: {'n_estimators': 2004, 'learning_rate': 0.010061292123123369, 'max_depth': 4, 'subsample': 0.9226048431612656, 'colsample_bytree': 0.9117094324101971}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:25:42] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:26:08] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:26:34] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:27:00,175] Trial 73 finished with value: 0.904525727199708 and parameters: {'n_estimators': 1732, 'learning_rate': 0.008268775812057266, 'max_depth': 4, 'subsample': 0.9319048931318168, 'colsample_bytree': 0.9083615934802256}. Best is trial 31 with value: 0.9057489874535184.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:27:00] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:27:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:27:49] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:28:14,139] Trial 74 finished with value: 0.9058133001990885 and parameters: {'n_estimators': 1629, 'learning_rate': 0.01579726149988962, 'max_depth': 4, 'subsample': 0.9352096081034901, 'colsample_bytree': 0.9191283363226377}. Best is trial 74 with value: 0.9058133001990885.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:28:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:28:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:28:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:29:12,514] Trial 75 finished with value: 0.9049119269777269 and parameters: {'n_estimators': 1660, 'learning_rate': 0.016899955705442428, 'max_depth': 3, 'subsample': 0.9282324553277438, 'colsample_bytree': 0.923022532253956}. Best is trial 74 with value: 0.9058133001990885.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:29:12] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:29:34] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:29:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:30:13,404] Trial 76 finished with value: 0.9058777372925811 and parameters: {'n_estimators': 1469, 'learning_rate': 0.015139821103617852, 'max_depth': 4, 'subsample': 0.944435813311348, 'colsample_bytree': 0.9320650473340371}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:30:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:30:36] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:30:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:31:20,523] Trial 77 finished with value: 0.9032382163742887 and parameters: {'n_estimators': 1288, 'learning_rate': 0.01575921239419109, 'max_depth': 5, 'subsample': 0.9446493276121292, 'colsample_bytree': 0.933658802237299}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:31:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:31:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:32:01] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:32:21,252] Trial 78 finished with value: 0.9055557632165864 and parameters: {'n_estimators': 1459, 'learning_rate': 0.018622340540954817, 'max_depth': 4, 'subsample': 0.9502248174527499, 'colsample_bytree': 0.9307495278439661}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:32:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:32:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:32:56] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:33:12,471] Trial 79 finished with value: 0.9046544646039784 and parameters: {'n_estimators': 1449, 'learning_rate': 0.029251886136777306, 'max_depth': 3, 'subsample': 0.939191428048049, 'colsample_bytree': 0.9308934092642669}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:33:12] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:33:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:34:09] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:34:37,527] Trial 80 finished with value: 0.9009847957556771 and parameters: {'n_estimators': 1388, 'learning_rate': 0.02229478168166032, 'max_depth': 6, 'subsample': 0.9454221605884056, 'colsample_bytree': 0.9280592886036917}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:34:37] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:34:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:35:22] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:35:43,613] Trial 81 finished with value: 0.9044613398453843 and parameters: {'n_estimators': 1570, 'learning_rate': 0.01883645630368898, 'max_depth': 4, 'subsample': 0.9426623547359667, 'colsample_bytree': 0.9357833200875674}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:35:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:36:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:36:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:36:46,932] Trial 82 finished with value: 0.9051051885190359 and parameters: {'n_estimators': 1489, 'learning_rate': 0.014690509495056651, 'max_depth': 4, 'subsample': 0.9505846997393279, 'colsample_bytree': 0.9202303773835389}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:36:47] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:37:14] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:37:41] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:38:07,588] Trial 83 finished with value: 0.9032380920263661 and parameters: {'n_estimators': 1887, 'learning_rate': 0.021838459819442376, 'max_depth': 4, 'subsample': 0.9400187900028688, 'colsample_bytree': 0.9406350383063592}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:38:07] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:38:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:38:42] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:38:59,789] Trial 84 finished with value: 0.9047188643930942 and parameters: {'n_estimators': 1269, 'learning_rate': 0.0131054908372697, 'max_depth': 4, 'subsample': 0.9493519762585094, 'colsample_bytree': 0.926199432706447}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:38:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:39:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:39:50] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:40:13,824] Trial 85 finished with value: 0.9016285822553675 and parameters: {'n_estimators': 1734, 'learning_rate': 0.03397803088757533, 'max_depth': 4, 'subsample': 0.9544338705650708, 'colsample_bytree': 0.9234720725814413}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:40:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:40:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:41:06] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:41:34,121] Trial 86 finished with value: 0.9014999194598506 and parameters: {'n_estimators': 1570, 'learning_rate': 0.01823200176266173, 'max_depth': 5, 'subsample': 0.9622927296397455, 'colsample_bytree': 0.9487571655224712}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:41:34] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:41:49] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:42:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:42:23,685] Trial 87 finished with value: 0.9047832144430411 and parameters: {'n_estimators': 1137, 'learning_rate': 0.01554801483821251, 'max_depth': 4, 'subsample': 0.9582197681071144, 'colsample_bytree': 0.9147860925555076}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:42:23] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:42:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:43:02] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:43:23,586] Trial 88 finished with value: 0.9008561080905757 and parameters: {'n_estimators': 1427, 'learning_rate': 0.006433153732658148, 'max_depth': 4, 'subsample': 0.9882875917514258, 'colsample_bytree': 0.930179016266795}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:43:23] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:43:41] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:44:02] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:44:19,979] Trial 89 finished with value: 0.9029805674786561 and parameters: {'n_estimators': 1326, 'learning_rate': 0.009694191809931453, 'max_depth': 4, 'subsample': 0.9375536568706417, 'colsample_bytree': 0.9193442552027232}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:44:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:44:37] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:44:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:45:13,114] Trial 90 finished with value: 0.904268202651998 and parameters: {'n_estimators': 1518, 'learning_rate': 0.026708455927001856, 'max_depth': 3, 'subsample': 0.9478556411421757, 'colsample_bytree': 0.9376205618118242}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:45:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:45:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:45:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:46:20,831] Trial 91 finished with value: 0.9044613647149689 and parameters: {'n_estimators': 1632, 'learning_rate': 0.012421500637140375, 'max_depth': 4, 'subsample': 0.9417981630638558, 'colsample_bytree': 0.9134389553255647}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:46:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:46:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:47:11] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:47:33,619] Trial 92 finished with value: 0.9052339507928909 and parameters: {'n_estimators': 1696, 'learning_rate': 0.01196754135641903, 'max_depth': 4, 'subsample': 0.9346376952539704, 'colsample_bytree': 0.9235736276596653}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:47:33] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:47:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:48:22] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:48:54,367] Trial 93 finished with value: 0.9056202624840403 and parameters: {'n_estimators': 1708, 'learning_rate': 0.014460613235833631, 'max_depth': 4, 'subsample': 0.9439211924133128, 'colsample_bytree': 0.9234195240213753}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:48:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:49:44] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:50:33] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:51:23,842] Trial 94 finished with value: 0.9004698088342186 and parameters: {'n_estimators': 2104, 'learning_rate': 0.013868156708322534, 'max_depth': 7, 'subsample': 0.9439370348136458, 'colsample_bytree': 0.9318601444728976}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:51:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:51:47] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:52:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:52:39,546] Trial 95 finished with value: 0.9049120015864806 and parameters: {'n_estimators': 1771, 'learning_rate': 0.016846142194893703, 'max_depth': 4, 'subsample': 0.9518587040285095, 'colsample_bytree': 0.9275398644265292}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:52:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:52:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:53:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:53:39,178] Trial 96 finished with value: 0.9016930442184447 and parameters: {'n_estimators': 1377, 'learning_rate': 0.007353975369830253, 'max_depth': 4, 'subsample': 0.9284299859842333, 'colsample_bytree': 0.944067392476222}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:53:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:54:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:54:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:54:59,628] Trial 97 finished with value: 0.9035600536675686 and parameters: {'n_estimators': 1837, 'learning_rate': 0.02101777321481836, 'max_depth': 4, 'subsample': 0.9466444036871593, 'colsample_bytree': 0.919024771324933}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:54:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:55:20] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:55:43] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:56:04,461] Trial 98 finished with value: 0.9046544397343937 and parameters: {'n_estimators': 1550, 'learning_rate': 0.00985003635727512, 'max_depth': 4, 'subsample': 0.938453920660845, 'colsample_bytree': 0.9248837283266185}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:56:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:56:40] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:57:13] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:57:46,357] Trial 99 finished with value: 0.9014998821554737 and parameters: {'n_estimators': 1952, 'learning_rate': 0.018364076300447264, 'max_depth': 5, 'subsample': 0.9364795776650877, 'colsample_bytree': 0.91012959689997}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:57:46] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:58:08] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:58:28] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:58:50,700] Trial 100 finished with value: 0.9052983381472144 and parameters: {'n_estimators': 1469, 'learning_rate': 0.013945564293874998, 'max_depth': 4, 'subsample': 0.9318732665411877, 'colsample_bytree': 0.9218760899809724}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:58:50] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:59:10] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:59:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 14:59:52,052] Trial 101 finished with value: 0.9052339507928909 and parameters: {'n_estimators': 1457, 'learning_rate': 0.01384433188850613, 'max_depth': 4, 'subsample': 0.9311256560067338, 'colsample_bytree': 0.9213024633651974}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:59:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:00:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:00:39] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:01:03,173] Trial 102 finished with value: 0.9056201754404944 and parameters: {'n_estimators': 1667, 'learning_rate': 0.015978987509927635, 'max_depth': 4, 'subsample': 0.9238999313290406, 'colsample_bytree': 0.9172846396479158}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:01:03] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:01:27] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:01:48] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:02:12,794] Trial 103 finished with value: 0.9051051636494513 and parameters: {'n_estimators': 1624, 'learning_rate': 0.01572157802523377, 'max_depth': 4, 'subsample': 0.924233582490545, 'colsample_bytree': 0.9169501750063735}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:02:12] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:02:38] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:03:00] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:03:24,901] Trial 104 finished with value: 0.9045257023301234 and parameters: {'n_estimators': 1684, 'learning_rate': 0.010601838802134817, 'max_depth': 4, 'subsample': 0.9409934088325058, 'colsample_bytree': 0.9159644621304334}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:03:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:03:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:04:19] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:04:45,812] Trial 105 finished with value: 0.9000835593170304 and parameters: {'n_estimators': 1782, 'learning_rate': 0.05531514474661433, 'max_depth': 4, 'subsample': 0.9173130964139692, 'colsample_bytree': 0.9125310862302709}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:04:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:05:06] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:05:30] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:05:54,348] Trial 106 finished with value: 0.9032380920263661 and parameters: {'n_estimators': 1578, 'learning_rate': 0.02437111885818864, 'max_depth': 4, 'subsample': 0.9340571239816625, 'colsample_bytree': 0.9291084565228518}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:05:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:06:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:06:38] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:06:58,923] Trial 107 finished with value: 0.9034312292197524 and parameters: {'n_estimators': 1522, 'learning_rate': 0.008290407574196502, 'max_depth': 4, 'subsample': 0.927674602811325, 'colsample_bytree': 0.9190291313455218}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:06:59] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:07:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:07:52] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:08:18,263] Trial 108 finished with value: 0.9048475893625724 and parameters: {'n_estimators': 2214, 'learning_rate': 0.0119965835223523, 'max_depth': 3, 'subsample': 0.9028002736722536, 'colsample_bytree': 0.9147203201307165}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:08:18] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:08:50] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:09:22] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:09:56,475] Trial 109 finished with value: 0.9031093421873034 and parameters: {'n_estimators': 2319, 'learning_rate': 0.019209875918229452, 'max_depth': 4, 'subsample': 0.9435775562135662, 'colsample_bytree': 0.9263468305895856}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:09:56] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:10:29] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:11:02] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:11:37,381] Trial 110 finished with value: 0.9037531784261627 and parameters: {'n_estimators': 2429, 'learning_rate': 0.016615882571868006, 'max_depth': 4, 'subsample': 0.925887110433885, 'colsample_bytree': 0.9228619814600848}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:11:37] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:11:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:12:18] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:12:38,379] Trial 111 finished with value: 0.9051052258234127 and parameters: {'n_estimators': 1467, 'learning_rate': 0.01423114329559625, 'max_depth': 4, 'subsample': 0.9307149993078571, 'colsample_bytree': 0.9193413455779299}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:12:38] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:13:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:13:26] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:13:51,061] Trial 112 finished with value: 0.9053627379363304 and parameters: {'n_estimators': 1663, 'learning_rate': 0.013431999622179976, 'max_depth': 4, 'subsample': 0.9204346741589355, 'colsample_bytree': 0.9213352453660679}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:13:51] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:14:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:14:36] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:15:00,874] Trial 113 finished with value: 0.9053627006319536 and parameters: {'n_estimators': 1643, 'learning_rate': 0.014991746203594008, 'max_depth': 4, 'subsample': 0.909967540583552, 'colsample_bytree': 0.9245092899068452}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:15:00] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:15:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:15:46] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:16:11,212] Trial 114 finished with value: 0.9049763889408041 and parameters: {'n_estimators': 1657, 'learning_rate': 0.020893761115369325, 'max_depth': 4, 'subsample': 0.9126332056968942, 'colsample_bytree': 0.9244715948547078}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:16:11] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:16:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:16:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:17:24,150] Trial 115 finished with value: 0.9048476764061183 and parameters: {'n_estimators': 1731, 'learning_rate': 0.015459744102816328, 'max_depth': 4, 'subsample': 0.915790009475912, 'colsample_bytree': 0.9327511537884331}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:17:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:17:53] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:18:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:18:49,998] Trial 116 finished with value: 0.9032380547219893 and parameters: {'n_estimators': 1590, 'learning_rate': 0.017006333251784144, 'max_depth': 5, 'subsample': 0.9055551746989567, 'colsample_bytree': 0.9169171405509186}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:18:50] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:19:17] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:19:44] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:20:10,185] Trial 117 finished with value: 0.905233913488514 and parameters: {'n_estimators': 1793, 'learning_rate': 0.012962652402228818, 'max_depth': 4, 'subsample': 0.9212979158725586, 'colsample_bytree': 0.9358682541295881}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:20:10] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:20:34] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:21:00] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:21:27,379] Trial 118 finished with value: 0.904590139423616 and parameters: {'n_estimators': 1851, 'learning_rate': 0.014971640690607815, 'max_depth': 4, 'subsample': 0.9092875110604943, 'colsample_bytree': 0.9103120114750485}. Best is trial 76 with value: 0.9058777372925811.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:21:27] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:21:46] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:22:07] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:22:26,199] Trial 119 finished with value: 0.9060708371815908 and parameters: {'n_estimators': 1399, 'learning_rate': 0.018343556445067386, 'max_depth': 4, 'subsample': 0.9229740607858694, 'colsample_bytree': 0.9206263171952277}. Best is trial 119 with value: 0.9060708371815908.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:22:26] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:22:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:23:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:23:24,346] Trial 120 finished with value: 0.9060064125228903 and parameters: {'n_estimators': 1387, 'learning_rate': 0.019078069929728738, 'max_depth': 4, 'subsample': 0.9203743138299247, 'colsample_bytree': 0.9283159432109788}. Best is trial 119 with value: 0.9060708371815908.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:23:24] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:23:45] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:24:03] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:24:21,575] Trial 121 finished with value: 0.9054270631166926 and parameters: {'n_estimators': 1363, 'learning_rate': 0.018822117351306553, 'max_depth': 4, 'subsample': 0.9210487931748008, 'colsample_bytree': 0.9284144933927668}. Best is trial 119 with value: 0.9060708371815908.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:24:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:24:42] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:25:00] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:25:18,879] Trial 122 finished with value: 0.9037531162522013 and parameters: {'n_estimators': 1353, 'learning_rate': 0.03114607940470598, 'max_depth': 4, 'subsample': 0.9183469471532997, 'colsample_bytree': 0.9284465304460094}. Best is trial 119 with value: 0.9060708371815908.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:25:19] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:25:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:26:26] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:26:58,327] Trial 123 finished with value: 0.8996972724954656 and parameters: {'n_estimators': 1224, 'learning_rate': 0.02292131799968925, 'max_depth': 8, 'subsample': 0.9198656338728688, 'colsample_bytree': 0.926817277524318}. Best is trial 119 with value: 0.9060708371815908.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:26:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:27:18] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:27:36] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:27:54,360] Trial 124 finished with value: 0.9053626881971614 and parameters: {'n_estimators': 1303, 'learning_rate': 0.019297330641990056, 'max_depth': 4, 'subsample': 0.9231059514738706, 'colsample_bytree': 0.9213841674642121}. Best is trial 119 with value: 0.9060708371815908.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:27:54] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:28:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:28:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:28:57,195] Trial 125 finished with value: 0.9060708869207597 and parameters: {'n_estimators': 1425, 'learning_rate': 0.017318177456778546, 'max_depth': 4, 'subsample': 0.9206296430073134, 'colsample_bytree': 0.914171048411071}. Best is trial 125 with value: 0.9060708869207597.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:28:57] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:29:16] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:29:35] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:29:56,208] Trial 126 finished with value: 0.9058133250686731 and parameters: {'n_estimators': 1410, 'learning_rate': 0.01760120303256749, 'max_depth': 4, 'subsample': 0.926503550629124, 'colsample_bytree': 0.9311020904604554}. Best is trial 125 with value: 0.9060708869207597.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:29:56] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:30:15] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:30:37] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:30:56,102] Trial 127 finished with value: 0.9038818287868873 and parameters: {'n_estimators': 1435, 'learning_rate': 0.02550649307118879, 'max_depth': 4, 'subsample': 0.9246865345299321, 'colsample_bytree': 0.9312334595518073}. Best is trial 125 with value: 0.9060708869207597.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:30:56] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:31:12] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:31:28] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:31:47,271] Trial 128 finished with value: 0.9050408011647123 and parameters: {'n_estimators': 1171, 'learning_rate': 0.017405916119557507, 'max_depth': 4, 'subsample': 0.915503552587035, 'colsample_bytree': 0.9718409241110664}. Best is trial 125 with value: 0.9060708869207597.
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:31:47] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:32:04] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:32:21] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# [I 2025-10-26 15:32:38,067] Trial 129 finished with value: 0.904139291160636 and parameters: {'n_estimators': 1508, 'learning_rate': 0.03929380474458222, 'max_depth': 3, 'subsample': 0.9268902839553017, 'colsample_bytree': 0.93976315413579}. Best is trial 125 with value: 0.9060708869207597.

# ===== Optuna Optimization Results =====
# Best CV Accuracy: 0.9061
# Best Parameters Found:
#   n_estimators: 1425
#   learning_rate: 0.017318177456778546
#   max_depth: 4
#   subsample: 0.9206296430073134
#   colsample_bytree: 0.914171048411071

# ✅ Saved all Optuna trials to 'optuna_xgb_results.csv'

# ===== Using Best Parameters for Final Training =====
# {'n_estimators': 1425, 'learning_rate': 0.017318177456778546, 'max_depth': 4, 'subsample': 0.9206296430073134, 'colsample_bytree': 0.914171048411071, 'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'mlogloss', 'n_jobs': -1}

# ===== Fold 1 =====
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:32:38] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# Validation Accuracy: 0.9051

# ===== Fold 2 =====
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:33:05] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# Validation Accuracy: 0.9051

# ===== Fold 3 =====
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:33:32] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# Validation Accuracy: 0.9079

# ===== Fold 4 =====
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:33:58] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# Validation Accuracy: 0.9089

# ===== Fold 5 =====
# /usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:34:25] WARNING: /workspace/src/learner.cc:790:
# Parameters: { "use_label_encoder" } are not used.

#   bst.update(dtrain, iteration=i, fobj=obj)
# Validation Accuracy: 0.9047

# ===== Final Results =====
# Mean CV Accuracy: 0.9063
# Std Dev: 0.0017

# ✅ Prediction Preview:
#       id       WeightCategory
# 0  15533     Obesity_Type_III
# 1  15534   Overweight_Level_I
# 2  15535  Overweight_Level_II
# 3  15536      Obesity_Type_II
# 4  15537        Normal_Weight

# Total rows: 5225

# ✅ Saved 'predictions.csv' — ready for Kaggle upload!